Week 5, Assignment C1:

Logistic Regression with Yahoo! Finance API in Python

In this notebook, we will study Logistic Regression using the Yahoo! Finance API in Python.

Please complete the lines bellow where you see "#TODO".

In [1]:
#Objective 1.
#Use this cell to import the Numpy (as np), Pandas (as pd), and YFinance (as yf) packages.

import numpy as np
import pandas as pd
import yfinance as yf
import sklearn
import warnings
warnings.filterwarnings("ignore")


#Our work will also require some components of the Sklearn and Pandas_Datareader packages as imported below:

from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from pandas_datareader import data as pdr
yf.pdr_override()

In [2]:
#Objective 2.
#Select a stock symbol for a stock whose historical data is available on the Yahoo! Finance website.  Store the
#string of your chosen symbol to the new variable "stock_symbol".

stock_symbol = 'AAPL'

In [3]:
#Objective 3.
#Use the function "pdr.get_data_yahoo(stock_symbol, start_date, end_date)" to generate a Pandas dataframe of
#historical stock data for your chosen stock.  Retain only the first four columns of the dataframe.  Be sure to 
#drop any rows containing NaN's, and take a peek at the resulting dataframe to make sure everything looks good.

df = pdr.get_data_yahoo(stock_symbol, start="2019-01-01", end="2021-01-01")
df = df[df.columns[0:4]]
df = df[['Open', 'High', 'Low', 'Close']]
df = df.dropna()
df

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close
Date,,,,
2018-12-31,39.632500,39.840000,39.119999,39.435001
2019-01-02,38.722500,39.712502,38.557499,39.480000
2019-01-03,35.994999,36.430000,35.500000,35.547501
2019-01-04,36.132500,37.137501,35.950001,37.064999
2019-01-07,37.174999,37.207500,36.474998,36.982498
...,...,...,...,...
2020-12-24,131.320007,133.460007,131.100006,131.970001
2020-12-28,133.990005,137.339996,133.509995,136.690002
2020-12-29,138.050003,138.789993,134.339996,134.869995


In [4]:
#Objective 4.
#Use the historical data to define some predictor variables.  Add these variables to the dataframe.  Include, at a
#minimum, (Predictor.I) the rolling average closing price over the past fifteen (15) days and (Predictor.II) the
#change in opening price over the past one (1) day.  Be sure to again drop any rows containing NaN's, and take a
#peek at the resulting dataframe to make sure everything looks good.  Store the dataframe as the new variable "X".

df['Close_15_Rolling'] = df['Close'].rolling(window = 15).mean()
df['Open_1_Change'] = df['Open'] .diff()
df = df.dropna()
X = df
X

,Open,High,Low,Close,Close_15_Rolling,Open_1_Change
Date,,,,,,
2019-01-22,39.102501,39.182499,38.154999,38.325001,38.136333,-0.272499
2019-01-23,38.537498,38.785000,37.924999,38.480000,38.072667,-0.565002
2019-01-24,38.527500,38.619999,37.935001,38.174999,37.985667,-0.009998
2019-01-25,38.869999,39.532501,38.580002,39.439999,38.245167,0.342499
2019-01-28,38.947498,39.082500,38.415001,39.075001,38.379167,0.077499
...,...,...,...,...,...,...
2020-12-24,131.320007,133.460007,131.100006,131.970001,126.245333,-0.839996
2020-12-28,133.990005,137.339996,133.509995,136.690002,127.208000,2.669998
2020-12-29,138.050003,138.789993,134.339996,134.869995,127.949333,4.059998


In [5]:
#Objective 5.
#Define the target or dependent variable to be one (1) if the change in closing price over the past one (1) day is
#nonnegative and negative one (-1) if the change in closing price over the past one (1) day is negative.  This
#variable's values should be forward-looking (i.e., you should subtract today's price from tomorrow's price rather
#than subtracting yesterday's price from today's price).  Store the resulting values as the new variable "y".  You
#may find the "np.where(*args)" function to be useful.  Look it up in Numpy documentation for support.

y = np.where((df.Close.shift(-1)-df['Close'])>=0,1,-1)

In [6]:
#Objective 6.
#Split the data into training and test sets, putting the first seventy percent (70%) of the data in the training
#set.

index = int(0.7*len(X))
X_train = X[:index]
X_test = X[index:]
y_train = y[:index]
y_test = y[index:]

In [7]:
#Objective 7.
#Instantiate the Logistic Regression model object, and use its ".fit(*args)" method to fit the model to the 
#training data.

logistic = LogisticRegression()
logistic = logistic.fit(X_train,y_train)

In [8]:
#Objective 8.
#Examine the model's coefficients by using its ".coef_" method.

#pd.DataFrame(zip(X.columns, np.transpose(logistic.coef_)))

pd.DataFrame({'column':X.columns.values,'b':logistic.coef_.ravel()})

,column,b
0,Open,0.444820
1,High,-0.127777
2,Low,0.090477
3,Close,-0.337460
4,Close_15_Rolling,-0.070364
5,Open_1_Change,-0.175884


In [9]:
#Objective 9.
#Use the model's ".predict_proba(*args)" and ".predict(*args)" methods to generate predictions over the test set.

probabilities = logistic.predict_proba(X_test)
predictions = logistic.predict(X_test)

In [10]:
#Objective 10.
#Use the function "metrics.confusion_matrix(*args)" to create a confusion matrix comparing the predicted and true
#classification labels over the test set.

sklearn.metrics.confusion_matrix(y_pred = predictions,y_true= y_test )

array([[30, 35],
       [33, 50]])

In [11]:
# Alternative method using report classfication 

print(sklearn.metrics.classification_report(y_pred = predictions,y_true= y_test))

              precision    recall  f1-score   support

          -1       0.48      0.46      0.47        65
           1       0.59      0.60      0.60        83

    accuracy                           0.54       148
   macro avg       0.53      0.53      0.53       148
weighted avg       0.54      0.54      0.54       148



In [12]:
#Objective 11.
#Calculate the model's accuracy on the test set using its ".score(*args)" method.

logistic.score(X_test,y_test)

0.5405405405405406

In [21]:
#Objective 12.
#Use five-fold cross validation to cross-check the accuracy of the model over different held-out test sets.  This
#is where you should use the function "cross_val_score(*args)".

cross_val= cross_val_score(LogisticRegression(), X, y, cv=5)
cross_val

array([0.54545455, 0.55555556, 0.56122449, 0.64285714, 0.58163265])

In [22]:
np.mean(cross_val)

0.5773448773448774